In [2]:
import pandas as pd

In [3]:
import numpy as np

In [4]:
data_cafe = pd.read_csv("dirty_cafe_sales.csv", encoding= "ISO-8859-1")

In [5]:
data_cafe

,Transaction ID,Item,Quantity,Price Per Unit,Total Spent,Payment Method,Location,Transaction Date
0,TXN_1961373,Coffee,2,2.0,4.0,Credit Card,Takeaway,2023-09-08
1,TXN_4977031,Cake,4,3.0,12.0,Cash,In-store,2023-05-16
2,TXN_4271903,Cookie,4,1.0,ERROR,Credit Card,In-store,2023-07-19
3,TXN_7034554,Salad,2,5.0,10.0,UNKNOWN,UNKNOWN,2023-04-27
4,TXN_3160411,Coffee,2,2.0,4.0,Digital Wallet,In-store,2023-06-11
...,...,...,...,...,...,...,...,...
9995,TXN_7672686,Coffee,2,2.0,4.0,NaN,UNKNOWN,2023-08-30
9996,TXN_9659401,NaN,3,NaN,3.0,Digital Wallet,NaN,2023-06-02
9997,TXN_5255387,Coffee,4,2.0,8.0,Digital Wallet,NaN,2023-03-02
9998,TXN_7695629,Cookie,3,NaN,3.0,Digital Wallet,NaN,2023-12-02


In [6]:
data=data_cafe.copy()

In [8]:
def porcentaje_valores_especificos(data):
    total_filas = len(data)
    
    porcentaje_nulos = data.isnull().sum() / total_filas * 100
    porcentaje_error = (data == "ERROR").sum() / total_filas * 100
    porcentaje_unknown = (data == "UNKNOWN").sum() / total_filas * 100
    
    resultados = pd.DataFrame({
        'Porcentaje de Nulos': porcentaje_nulos,
        'Porcentaje de ERROR': porcentaje_error,
        'Porcentaje de UNKNOWN': porcentaje_unknown
    })
    resultados = resultados[(resultados > 0).any(axis=1)]
    return resultados.sort_values(by='Porcentaje de ERROR', ascending=False)

In [95]:
porcentaje_valores_especificos(data)

,Porcentaje de Nulos,Porcentaje de ERROR,Porcentaje de UNKNOWN


In [68]:
data[data.get("Price Per Unit").isnull()].shape

(0, 6)

In [11]:
data["Transaction Date"].ffill(inplace=True)

C:\Users\ivan_\AppData\Local\Temp\ipykernel_21512\433038576.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  data["Transaction Date"].ffill(inplace=True)


In [32]:
data.drop("Payment Method", axis=1, inplace=True)

In [15]:
dropItemPrice = data["Item"].isnull() & data["Price Per Unit"].isnull()

In [16]:
data[data.get("Item").isnull() & data.get("Price Per Unit").isnull()].shape#Sacar total

(7, 8)

In [17]:
data.drop(index=data[dropItemPrice].index, inplace=True)

In [18]:
def eliminar_filas_valores_invalidos(data, colum1, colum2, colum3):
    col1_null = data[colum1].isnull()
    col2_valid = data[colum2].notnull() & ~data[colum2].astype(str).isin(["ERROR", "UNKNOWN"])
    col3_valid = data[colum3].notnull() & ~data[colum3].astype(str).isin(["ERROR", "UNKNOWN"])
    condicion = col1_null & col2_valid & col3_valid
    data.drop(index=data[condicion].index, inplace=True)

In [19]:
eliminar_filas_valores_invalidos(data,"Quantity","Price Per Unit","Total Spent")

In [69]:
data

,Transaction ID,Item,Quantity,Price Per Unit,Total Spent,Transaction Date
0,TXN_1961373,Coffee,2.0,2.0,4.0,2023-09-08
1,TXN_4977031,Cake,4.0,3.0,12.0,2023-05-16
2,TXN_4271903,Cookie,4.0,1.0,4.0,2023-07-19
3,TXN_7034554,Salad,2.0,5.0,10.0,2023-04-27
4,TXN_3160411,Coffee,2.0,2.0,4.0,2023-06-11
...,...,...,...,...,...,...
9994,TXN_7851634,UNKNOWN,4.0,4.0,16.0,2023-01-08
9995,TXN_7672686,Coffee,2.0,2.0,4.0,2023-08-30
9997,TXN_5255387,Coffee,4.0,2.0,8.0,2023-03-02
9998,TXN_7695629,Cookie,3.0,1.0,3.0,2023-12-02


In [42]:
data["Total Spent"] = pd.to_numeric(data["Total Spent"], errors='coerce')
data["Price Per Unit"] = pd.to_numeric(data["Price Per Unit"], errors='coerce')

data["Quantity"] = data["Quantity"].str.strip()
data["Quantity"] = data["Quantity"].replace([None, 'UNKNOWN', 'ERROR', 'nan'], float('nan'))

filas_invalidas_quantity = data[data["Quantity"].isna()]

if not filas_invalidas_quantity.empty:
    data.loc[data["Quantity"].isna(), "Quantity"] = data["Total Spent"] / data["Price Per Unit"]
    print(data.loc[data["Quantity"].isna(), ["Transaction ID", "Item", "Quantity", "Price Per Unit", "Total Spent", "Transaction Date"]])
else:
    print("No hay filas con 'ERROR', 'UNKNOWN' o 'NaN' en 'Quantity'.")

     Transaction ID      Item Quantity  Price Per Unit  Total Spent  \
236     TXN_8562645     Salad      NaN             5.0          NaN   
278     TXN_3229409     Juice      NaN             3.0          NaN   
629     TXN_9289174      Cake      NaN             NaN         12.0   
641     TXN_2962976     Juice      NaN             3.0          NaN   
738     TXN_8696094  Sandwich      NaN             4.0          NaN   
912     TXN_1575608  Sandwich      NaN             NaN         20.0   
1008    TXN_7225428       Tea      NaN             NaN          3.0   
1436    TXN_7590801       Tea      NaN             NaN          6.0   
1482    TXN_3593060  Smoothie      NaN             NaN         16.0   
2330    TXN_3849488     Salad      NaN             NaN          5.0   
2796    TXN_9188692      Cake      NaN             3.0          NaN   
3203    TXN_4565754  Smoothie      NaN             4.0          NaN   
3224    TXN_6297232    Coffee      NaN             2.0          NaN   
3401  

In [48]:
# Asegurarse de que "Total Spent" y "Quantity" sean numéricos (convertir a float)
data["Total Spent"] = pd.to_numeric(data["Total Spent"], errors='coerce')
data["Quantity"] = pd.to_numeric(data["Quantity"], errors='coerce')

# Limpiar posibles errores en los datos de "Price Per Unit", solo si son cadenas
data["Price Per Unit"] = pd.to_numeric(data["Price Per Unit"], errors='coerce')

# Filtrar las filas donde "Price Per Unit" sea "ERROR", "UNKNOWN" o NaN
filas_invalidas_price = data[data["Price Per Unit"].isna()]

# Verificar si existen filas con "ERROR", "UNKNOWN" o NaN en "Price Per Unit"
if not filas_invalidas_price.empty:
    # Realizar la operación Total Spent / Quantity y asignarlo a "Price Per Unit"
    data.loc[data["Price Per Unit"].isna(), "Price Per Unit"] = data["Total Spent"] / data["Quantity"]

    # Mostrar las columnas requeridas para las filas que fueron modificadas
    print(data.loc[data["Price Per Unit"].isna(), ["Transaction ID", "Item", "Quantity", "Price Per Unit", "Total Spent", "Transaction Date"]])
else:
    print("No hay filas con 'ERROR', 'UNKNOWN' o 'NaN' en 'Price Per Unit'.")


     Transaction ID      Item  Quantity  Price Per Unit  Total Spent  \
65      TXN_4987129  Sandwich       3.0             NaN          NaN   
629     TXN_9289174      Cake       NaN             NaN         12.0   
912     TXN_1575608  Sandwich       NaN             NaN         20.0   
1008    TXN_7225428       Tea       NaN             NaN          3.0   
1436    TXN_7590801       Tea       NaN             NaN          6.0   
1482    TXN_3593060  Smoothie       NaN             NaN         16.0   
1674    TXN_9367492       Tea       2.0             NaN          NaN   
1761    TXN_3611851       NaN       4.0             NaN          NaN   
2229    TXN_8498613  Sandwich       2.0             NaN          NaN   
2289    TXN_7524977   UNKNOWN       4.0             NaN          NaN   
2330    TXN_3849488     Salad       NaN             NaN          5.0   
2585    TXN_1259340       Tea       3.0             NaN          NaN   
3162    TXN_3577949      Cake       3.0             NaN         

In [59]:
# Asegurarse de que las columnas relevantes sean numéricas
data["Total Spent"] = pd.to_numeric(data["Total Spent"], errors='coerce')
data["Quantity"] = pd.to_numeric(data["Quantity"], errors='coerce')
data["Price Per Unit"] = pd.to_numeric(data["Price Per Unit"], errors='coerce')

# -----------------------------
# Rellenar valores nulos en Price Per Unit
# -----------------------------
filas_invalidas_price = data[data["Price Per Unit"].isna()]

if not filas_invalidas_price.empty:
    data.loc[data["Price Per Unit"].isna(), "Price Per Unit"] = (
        data["Total Spent"] / data["Quantity"]
    )

# -----------------------------
# Rellenar valores nulos en Quantity
# -----------------------------
filas_invalidas_quantity = data[data["Quantity"].isna()]

if not filas_invalidas_quantity.empty:
    data.loc[data["Quantity"].isna(), "Quantity"] = (
        data["Total Spent"] / data["Price Per Unit"]
    )

# -----------------------------
# Recalcular Total Spent para asegurar consistencia
# -----------------------------
data["Total Spent"] = data["Quantity"] * data["Price Per Unit"]

# Mostrar filas que todavía tengan Price Per Unit o Quantity nulas
print(
    data[
        data["Price Per Unit"].isna() | data["Quantity"].isna()
    ][["Transaction ID", "Item", "Quantity", "Price Per Unit", "Total Spent", "Transaction Date"]]
)


     Transaction ID      Item  Quantity  Price Per Unit  Total Spent  \
65      TXN_4987129  Sandwich       3.0             NaN          NaN   
236     TXN_8562645     Salad       NaN             5.0          NaN   
278     TXN_3229409     Juice       NaN             3.0          NaN   
629     TXN_9289174      Cake       NaN             NaN          NaN   
641     TXN_2962976     Juice       NaN             3.0          NaN   
738     TXN_8696094  Sandwich       NaN             4.0          NaN   
912     TXN_1575608  Sandwich       NaN             NaN          NaN   
1008    TXN_7225428       Tea       NaN             NaN          NaN   
1436    TXN_7590801       Tea       NaN             NaN          NaN   
1482    TXN_3593060  Smoothie       NaN             NaN          NaN   
1674    TXN_9367492       Tea       2.0             NaN          NaN   
1761    TXN_3611851       NaN       4.0             NaN          NaN   
2229    TXN_8498613  Sandwich       2.0             NaN         

In [66]:
data[data.get("Price Per Unit").isnull()]


,Transaction ID,Item,Quantity,Price Per Unit,Total Spent,Transaction Date


In [64]:
# Contamos cuántos nulos hay por fila entre las columnas seleccionadas
nulos_por_fila = data[["Quantity", "Price Per Unit", "Total Spent"]].isnull().sum(axis=1)

# Creamos una máscara para identificar filas con 2 o más nulos
filas_con_demasiados_nulos = nulos_por_fila >= 2

# Eliminamos esas filas del DataFrame
data.drop(index=data[filas_con_demasiados_nulos].index, inplace=True)


In [73]:
# Primero identificamos las filas con "UNKNOWN" o "ERROR"
condicion = data["Transaction Date"].isin(["UNKNOWN", "ERROR"])

# Las convertimos en NaN temporalmente para poder usar ffill()
data.loc[condicion, "Transaction Date"] = pd.NA

# Aplicamos el forward fill (rellenar hacia adelante)
data["Transaction Date"].fillna(method="ffill", inplace=True)


C:\Users\ivan_\AppData\Local\Temp\ipykernel_21512\1622921756.py:8: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  data["Transaction Date"].fillna(method="ffill", inplace=True)
C:\Users\ivan_\AppData\Local\Temp\ipykernel_21512\1622921756.py:8: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  data["Transaction Date"].fillna(method="ffill", inplace=True)


In [174]:
data

,Transaction ID,Item,Quantity,Price Per Unit,Total Spent,Transaction Date,Weeks Days,Calculated Total
0,TXN_1961373,Coffee,2.0,2.0,4.0,2023-09-08,Viernes,4.0
1,TXN_4977031,Cake,4.0,3.0,12.0,2023-05-16,Martes,12.0
2,TXN_4271903,Cookie,4.0,1.0,4.0,2023-07-19,Miércoles,4.0
3,TXN_7034554,Salad,2.0,5.0,10.0,2023-04-27,Jueves,10.0
4,TXN_3160411,Coffee,2.0,2.0,4.0,2023-06-11,Domingo,4.0
...,...,...,...,...,...,...,...,...
9994,TXN_7851634,Sandwich,4.0,4.0,16.0,2023-01-08,Domingo,16.0
9995,TXN_7672686,Coffee,2.0,2.0,4.0,2023-08-30,Miércoles,4.0
9997,TXN_5255387,Coffee,4.0,2.0,8.0,2023-03-02,Jueves,8.0
9998,TXN_7695629,Cookie,3.0,1.0,3.0,2023-12-02,Sábado,3.0


In [104]:
data["Quantity"].unique()

array([2., 4., 5., 3., 1.])

In [94]:
# Filtrar las filas donde "Item" o "Price Per Unit" son "UNKNOWN", "ERROR" o NaN
data_filtrada = data[~data["Item"].isin(["UNKNOWN", "ERROR"]) & data["Item"].notna()]

# Mostrar las combinaciones únicas de "Item" y "Price Per Unit"
data[["Item", "Price Per Unit"]].drop_duplicates()


,Item,Price Per Unit
0,Coffee,2.0
1,Cake,3.0
2,Cookie,1.0
3,Salad,5.0
5,Smoothie,4.0
6,Juice,3.0
7,Sandwich,4.0
14,Tea,1.5


In [92]:
# Diccionario que mapea los precios a los items correspondientes
precio_a_item = {
    2.0: "Coffee",
    3.0: "Cake",
    1.0: "Cookie",
    5.0: "Salad",
    4.0: "Smoothie",
    4.0: "Sandwich",
    3.0: "Juice",
    1.5: "Tea"
}

# Filtrar las filas donde "Item" sea "UNKNOWN", "ERROR" o NaN
condicion = data["Item"].isin(["UNKNOWN", "ERROR"]) | data["Item"].isna()

# Reemplazar los valores de "Item" según el "Price Per Unit"
for precio, item in precio_a_item.items():
    data.loc[condicion & (data["Price Per Unit"] == precio), "Item"] = item

# Verificar el cambio
print(data[data["Item"].isin(["UNKNOWN", "ERROR"]) | data["Item"].isna()])


Empty DataFrame
Columns: [Transaction ID, Item, Quantity, Price Per Unit, Total Spent, Transaction Date]
Index: []


In [121]:
data["Transaction Date"] = pd.to_datetime(data["Transaction Date"], errors='coerce')

In [122]:
print(data["Transaction Date"].dtype)

datetime64[ns]


In [178]:
data

,Transaction ID,Item,Quantity,Price Per Unit,Total Spent,Transaction Date,Weeks Days
0,TXN_1961373,Coffee,2.0,2.0,4.0,2023-09-08,Viernes
1,TXN_4977031,Cake,4.0,3.0,12.0,2023-05-16,Martes
2,TXN_4271903,Cookie,4.0,1.0,4.0,2023-07-19,Miércoles
3,TXN_7034554,Salad,2.0,5.0,10.0,2023-04-27,Jueves
4,TXN_3160411,Coffee,2.0,2.0,4.0,2023-06-11,Domingo
...,...,...,...,...,...,...,...
9994,TXN_7851634,Sandwich,4.0,4.0,16.0,2023-01-08,Domingo
9995,TXN_7672686,Coffee,2.0,2.0,4.0,2023-08-30,Miércoles
9997,TXN_5255387,Coffee,4.0,2.0,8.0,2023-03-02,Jueves
9998,TXN_7695629,Cookie,3.0,1.0,3.0,2023-12-02,Sábado


In [128]:
data["dias"] = data["Transaction Date"].dt.day_name()

In [148]:
dias_traducidos = {
    "Monday": "Lunes",
    "Tuesday": "Martes",
    "Wednesday": "Miércoles",
    "Thursday": "Jueves",
    "Friday": "Viernes",
    "Saturday": "Sábado",
    "Sunday": "Domingo"
}

In [152]:
data["Weeks Days"]=data["Week Day"].map(dias_traducidos)

In [177]:
data.drop("Calculated Total", axis=1, inplace=True)

In [163]:
data.get("Transaction ID").shape

(9811,)

In [179]:
resumen = data.groupby('Item').agg({
    'Quantity': 'sum',  # Volumen total de ventas
    'Total Spent': 'sum'
}).reset_index()

# Calcular el precio promedio por unidad
resumen['Precio Promedio'] = resumen['Total Spent'] / resumen['Quantity']

# Ordenar por volumen alto
top_volumen = resumen.sort_values(by='Quantity', ascending=False)

# Ordenar por precio promedio alto
top_precio = resumen.sort_values(by='Precio Promedio', ascending=False)

# Mostrar los primeros productos de cada uno (por ejemplo, top 5)
print("🔝 Productos con más volumen vendido:")
print(top_volumen.head())

print("\n💰 Productos con mejor precio promedio:")
print(top_precio.head())

🔝 Productos con más volumen vendido:
       Item  Quantity  Total Spent  Precio Promedio
3     Juice    4220.0      12660.0              3.0
5  Sandwich    4047.0      16188.0              4.0
1    Coffee    3842.0       7684.0              2.0
4     Salad    3775.0      18875.0              5.0
7       Tea    3596.0       5394.0              1.5

💰 Productos con mejor precio promedio:
       Item  Quantity  Total Spent  Precio Promedio
4     Salad    3775.0      18875.0              5.0
5  Sandwich    4047.0      16188.0              4.0
6  Smoothie    3307.0      13228.0              4.0
0      Cake    3400.0      10200.0              3.0
3     Juice    4220.0      12660.0              3.0


In [156]:
data.to_csv("cafe_clear.csv", index=False)

In [180]:
print(resumen[['Item', 'Quantity']])

       Item  Quantity
0      Cake    3400.0
1    Coffee    3842.0
2    Cookie    3524.0
3     Juice    4220.0
4     Salad    3775.0
5  Sandwich    4047.0
6  Smoothie    3307.0
7       Tea    3596.0


In [181]:
print(resumen[['Item', 'Quantity']].sort_values('Quantity', ascending=False))


       Item  Quantity
3     Juice    4220.0
5  Sandwich    4047.0
1    Coffee    3842.0
4     Salad    3775.0
7       Tea    3596.0
2    Cookie    3524.0
0      Cake    3400.0
6  Smoothie    3307.0
